In [25]:
%pip install gym
%pip install pyreadline
%config Completer.use_jedi = False
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy
import random
from gym import wrappers
import torch.optim as optim
import math
from torch.autograd import Variable

%matplotlib inline
%config IPCompleter.greedy=True

Looking in indexes: https://pypi.douban.com/simple/
You should consider upgrading via the '/home/yan/myProjects/dqn/dqn_pytorch_cartpole/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.douban.com/simple/
     |████████████████████████████████| 109 kB 1.3 MB/s eta 0:00:01
  Created wheel for pyreadline: filename=pyreadline-2.1-py3-none-any.whl size=93833 sha256=fbb3bab4ffd8c460c8fac5e615f0a771528bb3b517e0df7b00cc3ef646eb24fb
  Stored in directory: /home/yan/.cache/pip/wheels/79/8c/4f/6bc9a8aa98e0cad9e4af06bb0fcb2c8e31afebb4c71edb13a7
Successfully built pyreadline
You should consider upgrading via the '/home/yan/myProjects/dqn/dqn_pytorch_cartpole/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
Env_Name='CartPole-v0'

Batch_Size=3
Replay_Memory_Size=10000
Target_Network_Update_Frequency=10000
Discount_Factor=0.99
Learning_Rate=0.00025
Initial_Exploration=1
Final_Exploration=0.1
Exploration_Decay=200
Episodes=500
Hidden_Layer_Size=256


device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device=",device)

env=gym.make(Env_Name)
#env=wrappers.Monitor(env,'./tmp/cartpole-v1-1',force=True)
#observation=env.reset()

Input_Shape=env.observation_space.shape[0]
Action_Shape=env.action_space.n
#print('Input Shape=',Input_Shape)
#print('Action Shape=',Action_Shape)


device= cpu


/home/yan/myProjects/dqn/dqn_pytorch_cartpole/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:

class ReplayMemory:
    def __init__(self,capacity):
        self.capacity=capacity
        self.memory=[]
    def push(self,transition):
        self.memory.append(transition)
        if len(self.memory)>self.capacity:
            del self.memory[0]
    def sample(self,batch_size):
        return random.sample(self.memory,batch_size)

    def __len__(self):
        return len(self.memory)

In [4]:

class DQN(nn.Module):
    def __init__(self,input_shape=4,hidden_layer_shape=Hidden_Layer_Size,action_shape=2):
        super(DQN,self).__init__()
        self.l1=nn.Linear(input_shape,hidden_layer_shape)
        self.l2=nn.Linear(hidden_layer_shape,action_shape)
    def forward(self,x):
        x=F.relu(self.l1(x))
        x=self.l2(x)
        return x


In [5]:

model=DQN(input_shape=Input_Shape,hidden_layer_shape=Hidden_Layer_Size,action_shape=Action_Shape).to(device)


memory=ReplayMemory(Replay_Memory_Size)
optimizer=optim.Adam(model.parameters(),Learning_Rate)
steps_done=0
episode_durations=[]


In [6]:
def select_action(state):
    global steps_done
    sample=random.random()
    eps_threshold=Final_Exploration+(Initial_Exploration-Final_Exploration)*math.exp(-1.*steps_done/
                                                                                     Exploration_Decay)
    steps_done+=1
    if sample>eps_threshold:
        return  model(torch.tensor(state,dtype=torch.float32).to(device)).detach().max(1)[1].view(1,1)
    else:
        return torch.tensor([[random.randrange(2)]],dtype=torch.int64).to(device)

In [7]:

def run_episode(e,env):
    state=env.reset()
    steps=0
    while True:
        env.render()
        action=select_action(torch.tensor([state],dtype=torch.float32))
        next_state,reward,done,_=env.step(action[0,0].item())

        if done:
            reward=-1

        memory.push((torch.tensor([state],dtype=torch.float32).to(device),
                    torch.tensor(action,dtype=torch.int64).to(device),
                    torch.tensor([next_state],dtype=torch.float32).to(device),
                    torch.tensor([reward],dtype=torch.float32).to(device)))
        #learn()
        state=next_state
        steps+=1
        if done:
            print("{2} Episode {0} finished after {1} steps".format(e,steps,'\033[92m' if steps>=195
                                                                    else '\033[99m'))
            episode_durations.append(steps)
           # plot_duration()
            break


In [8]:
Replay_Memory_Size
Batch_Size
memory.memory=[]

In [9]:
for e in range(1):
    run_episode(e,env)

 Episode 0 finished after 10 steps


/home/yan/myProjects/dqn/dqn_pytorch_cartpole/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/home/yan/myProjects/dqn/dqn_pytorch_cartpole/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


memory.memory


In [10]:
transitions=memory.sample(Batch_Size)
transitions

[(tensor([[ 0.1467,  0.9922, -0.1857, -1.5949]]),
  tensor([[0]]),
  tensor([[ 0.1666,  0.7997, -0.2175, -1.3654]]),
  tensor([-1.])),
 (tensor([[ 0.0361,  0.7925, -0.0183, -1.1779]]),
  tensor([[1]]),
  tensor([[ 0.0520,  0.9879, -0.0418, -1.4762]]),
  tensor([1.])),
 (tensor([[ 0.0161,  0.4024,  0.0114, -0.5959]]),
  tensor([[1]]),
  tensor([[ 2.4198e-02,  5.9739e-01, -5.6727e-04, -8.8499e-01]]),
  tensor([1.]))]

In [11]:
batch_state,batch_action,batch_next_state,batch_reward=zip(*transitions)
batch_state,batch_action,batch_next_state,batch_reward

((tensor([[ 0.1467,  0.9922, -0.1857, -1.5949]]),
  tensor([[ 0.0361,  0.7925, -0.0183, -1.1779]]),
  tensor([[ 0.0161,  0.4024,  0.0114, -0.5959]])),
 (tensor([[0]]), tensor([[1]]), tensor([[1]])),
 (tensor([[ 0.1666,  0.7997, -0.2175, -1.3654]]),
  tensor([[ 0.0520,  0.9879, -0.0418, -1.4762]]),
  tensor([[ 2.4198e-02,  5.9739e-01, -5.6727e-04, -8.8499e-01]])),
 (tensor([-1.]), tensor([1.]), tensor([1.])))

In [12]:
batch_state=torch.tensor(torch.cat(batch_state))
batch_action=torch.tensor(torch.cat((batch_action)))
batch_next_state=torch.tensor(torch.cat((batch_next_state)))
batch_reward=torch.tensor(torch.cat((batch_reward)))
print("batch_state=",batch_state)
print("batch_action=",batch_action)
print("batch_next_state=",batch_next_state)
print("batch_reward=",batch_reward)

batch_state= tensor([[ 0.1467,  0.9922, -0.1857, -1.5949],
        [ 0.0361,  0.7925, -0.0183, -1.1779],
        [ 0.0161,  0.4024,  0.0114, -0.5959]])
batch_action= tensor([[0],
        [1],
        [1]])
batch_next_state= tensor([[ 1.6656e-01,  7.9974e-01, -2.1755e-01, -1.3654e+00],
        [ 5.1996e-02,  9.8788e-01, -4.1824e-02, -1.4762e+00],
        [ 2.4198e-02,  5.9739e-01, -5.6727e-04, -8.8499e-01]])
batch_reward= tensor([-1.,  1.,  1.])


/home/yan/myProjects/dqn/dqn_pytorch_cartpole/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/home/yan/myProjects/dqn/dqn_pytorch_cartpole/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/home/yan/myProjects/dqn/dqn_pytorch_cartpole/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid 

In [13]:
model_output=model(batch_state)
print("model_output=",model_output)
print("batch_action=",batch_action)
#print("q=",current_q_values.gather(1,batch_action))
#print("current_q_values=",current_q_values)

model_output= tensor([[-0.2217,  0.0603],
        [-0.1772,  0.0315],
        [-0.1441,  0.0161]], grad_fn=<AddmmBackward>)
batch_action= tensor([[0],
        [1],
        [1]])


In [14]:
current_q_values=torch.gather(model_output,1,batch_action)

In [15]:
print("current_q_values=",current_q_values)

current_q_values= tensor([[-0.2217],
        [ 0.0315],
        [ 0.0161]], grad_fn=<GatherBackward>)


In [16]:
model_output

tensor([[-0.2217,  0.0603],
        [-0.1772,  0.0315],
        [-0.1441,  0.0161]], grad_fn=<AddmmBackward>)

In [17]:
currentq=model_output.gather(1,batch_action)

In [18]:
currentq

tensor([[-0.2217],
        [ 0.0315],
        [ 0.0161]], grad_fn=<GatherBackward>)

In [19]:
batch_state

tensor([[ 0.1467,  0.9922, -0.1857, -1.5949],
        [ 0.0361,  0.7925, -0.0183, -1.1779],
        [ 0.0161,  0.4024,  0.0114, -0.5959]])

In [20]:
model(batch_state).detach()


tensor([[-0.2217,  0.0603],
        [-0.1772,  0.0315],
        [-0.1441,  0.0161]])

In [21]:
model(batch_state).detach().max(1)

torch.return_types.max(
values=tensor([0.0603, 0.0315, 0.0161]),
indices=tensor([1, 1, 1]))

In [22]:
model(batch_state).detach().max(1)[0]

tensor([0.0603, 0.0315, 0.0161])

In [23]:
model

DQN(
  (l1): Linear(in_features=4, out_features=256, bias=True)
  (l2): Linear(in_features=256, out_features=2, bias=True)
)

In [24]:
model

NameError: name 'mo' is not defined